<a href="https://colab.research.google.com/github/pen1064/Practice/blob/main/pytorch_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import sklearn
from torch import nn
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split

In [2]:
EPOCHES = 10
BATCHSIZE = 64

In [3]:
device = torch.device('cpu')

In [54]:
inFeatures = 4 #4D vector for each observation
hiddenDim = 8
nbClasses = 3
model = torch.nn.Sequential(nn.Linear(inFeatures, hiddenDim), 
                            nn.ReLU(),
                            nn.Linear(hiddenDim, nbClasses))

In [19]:
def create_batches(XX, yy, batchSize):
  for i in range(0, XX.shape[0],batchSize):
    yield(XX[i:i+batchSize], yy[i:i+batchSize])

In [87]:
(X, y) = make_blobs(n_samples=1000, n_features=4, centers=3,cluster_std=2.5, random_state=95)
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.8, random_state=22)

In [88]:
# convert to tensor
Xtrain = torch.from_numpy(Xtrain).float()
ytrain = torch.from_numpy(ytrain).long()
Xtest = torch.from_numpy(Xtest).float()
ytest = torch.from_numpy(ytest).long()

In [89]:
# put the model to device
model = model.to(device)

In [105]:
lr = 1e-2
opt = torch.optim.SGD(model.parameters(), lr = lr)
lossfn = nn.CrossEntropyLoss()
Train_Losses = []
Train_Accuracies = []
for epoch in range(EPOCHES):
  train_loss = 0
  train_acc = 0
  num_sample = 0
  model.train()
  for (xbatch, ybatch) in create_batches(Xtrain, ytrain, BATCHSIZE):
    (xbatch, ybatch) = (xbatch.to(device), ybatch.to(device))
    y_pred = model(xbatch)
    loss = lossfn(y_pred, ybatch)

    opt.zero_grad() #zero the gradient
    loss.backward() #calculate gradient and stuff
    opt.step() #update the weight

    train_loss += loss.item()*ybatch.size(0)
    train_acc += (y_pred.max(1)[1] == ybatch).sum().item()
    num_sample += ybatch.size(0)
    
  Train_Losses.append(train_loss)
  Train_Accuracies.append(train_acc)
  trainTemplate = "epoch: {} train loss: {:.3f} train accuracy: {:.3f}"
  print(trainTemplate.format(epoch, train_loss/num_sample, train_acc/num_sample))

  # Test after each epoch
  model.eval()
  test_acc = 0
  test_loss = 0
  test_num_sample = 0
  with torch.no_grad():
    for (Xtest_batch, ytest_batch) in create_batches(Xtest, ytest, BATCHSIZE):
      Xtest_batch, ytest_batch = Xtest_batch.to(device), ytest_batch.to(device)
      ytest_pred = model(Xtest_batch)
      test_loss = lossfn(ytest_pred, ytest_batch)

      test_acc += (ytest_pred.max(1)[1] == ytest_batch).sum().item()
      test_loss += test_loss.item()*ytest_batch.size(0)
      test_num_sample += ytest_batch.size(0)
    testTemplate = "epoch: {} test loss: {:.3f} test accuracy: {:.3f}"
    print(testTemplate.format(epoch, test_loss/test_num_sample, test_acc/test_num_sample))


epoch: 0 train loss: 0.046 train accuracy: 0.990
epoch: 0 test loss: 0.001 test accuracy: 0.995
epoch: 1 train loss: 0.046 train accuracy: 0.990
epoch: 1 test loss: 0.001 test accuracy: 0.995
epoch: 2 train loss: 0.046 train accuracy: 0.990
epoch: 2 test loss: 0.001 test accuracy: 0.995
epoch: 3 train loss: 0.046 train accuracy: 0.990
epoch: 3 test loss: 0.001 test accuracy: 0.995
epoch: 4 train loss: 0.045 train accuracy: 0.990
epoch: 4 test loss: 0.001 test accuracy: 0.995
epoch: 5 train loss: 0.045 train accuracy: 0.990
epoch: 5 test loss: 0.001 test accuracy: 0.995
epoch: 6 train loss: 0.045 train accuracy: 0.990
epoch: 6 test loss: 0.000 test accuracy: 0.995
epoch: 7 train loss: 0.045 train accuracy: 0.990
epoch: 7 test loss: 0.000 test accuracy: 0.995
epoch: 8 train loss: 0.045 train accuracy: 0.990
epoch: 8 test loss: 0.000 test accuracy: 0.995
epoch: 9 train loss: 0.045 train accuracy: 0.990
epoch: 9 test loss: 0.000 test accuracy: 0.995
